<a href="https://colab.research.google.com/github/ferygood/antifungal-linguist/blob/main/ipynb/t5_test_drug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[torch] tokenizers datasets evaluate rouge_score sentencepiece huggingface_hub pandas openpyxl --upgrade

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import nltk
from datasets import load_dataset
import evaluate
import numpy as np
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
import pandas as pd
from datasets import Dataset, DatasetDict

In [ ]:
def read_xlsx_drug_data(file_path):

  xls = pd.ExcelFile(file_path, engine="openpyxl")

  dataframes = {}

  for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name, engine="openpyxl")
    dataframes[sheet_name] = df

  return dataframes


def convert_dataframe_to_dataset(dataframes):

  dataset_dict = {}

  for split, df in dataframes.items():
    dataset_dict[split] = Dataset.from_pandas(df)

  dataset_dict = DatasetDict(dataset_dict)
  return dataset_dict

file_path = "drug_table_v3.xlsx"
result = read_xlsx_drug_data(file_path)
dataset = convert_dataframe_to_dataset(result)
print (dataset)

DatasetDict({
    train: Dataset({
        features: ['Question', 'Answer'],
        num_rows: 10
    })
    test: Dataset({
        features: ['Question', 'Answer'],
        num_rows: 10
    })
    valid: Dataset({
        features: ['Question', 'Answer'],
        num_rows: 10
    })
})


In [ ]:
from datasets import Dataset, concatenate_datasets

# Define the target dataset sizes
train_size = 16000
valid_size = 1000
test_size = 1000

# Repeat rows to achieve the desired dataset sizes
repeated_dataset = concatenate_datasets([dataset["train"]] * (train_size // 12))
repeated_dataset = repeated_dataset.shuffle(seed=42)

valid_dataset = concatenate_datasets([dataset["valid"]] * (valid_size // 12))
valid_dataset = valid_dataset.shuffle(seed=42)

test_dataset = concatenate_datasets([dataset["test"]] * (test_size // 12))
test_dataset = test_dataset.shuffle(seed=42)

# You can now use 'repeated_dataset', 'valid_dataset', and 'test_dataset' as per your requirements.

In [ ]:
# Create a new DatasetDict with the combined datasets
combined_dataset = DatasetDict({
    "train": repeated_dataset,
    "valid": valid_dataset,
    "test": test_dataset
})

print(combined_dataset)

DatasetDict({
    train: Dataset({
        features: ['Question', 'Answer'],
        num_rows: 13330
    })
    valid: Dataset({
        features: ['Question', 'Answer'],
        num_rows: 830
    })
    test: Dataset({
        features: ['Question', 'Answer'],
        num_rows: 830
    })
})


In [ ]:
base = "t5-base"
model = T5ForConditionalGeneration.from_pretrained(base)
tokenizer = T5Tokenizer.from_pretrained(base)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the

In [ ]:
# prefix our task
prefix1 = ""
prefix2 = ""

# define preprocessing function
def preprocess_function(examples):
  """Add prefix to the sentences, tokenize the text, and set the labels"""
  # the "inputs" are the tokenized answer
  inputs = [prefix1 + doc for doc in examples["Question"]]
  model_inputs = tokenizer(inputs, max_length=1280, truncation=True)

  # the "labels" are the tokenized outputs
  labels = [prefix2 + doc for doc in examples["Answer"]]
  labels_token = tokenizer(labels, max_length=1280, truncation=True)
  model_inputs["labels"] = labels_token["input_ids"]
  return model_inputs

# Map the preprocessing function across our datasets
tokenized_dataset = combined_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/13330 [00:00<?, ? examples/s]

Map:   0%|          | 0/830 [00:00<?, ? examples/s]

Map:   0%|          | 0/830 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_dataset["train"]["Question"][0])
print(tokenized_dataset["train"]["Answer"][0])
print(tokenized_dataset["train"]["input_ids"][0])
print(tokenized_dataset["train"]["attention_mask"][0])
print(tokenized_dataset["train"]["labels"][0])

[Label] Is amphotericin associated with antifungal medicine and Candida auris?
antifungal medicine, Candida auris
[784, 434, 10333, 908, 27, 7, 183, 9553, 449, 1294, 29, 1968, 28, 1181, 9940, 138, 4404, 11, 25833, 3, 9, 459, 7, 58, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1181, 9940, 138, 4404, 6, 25833, 3, 9, 459, 7, 1]


In [ ]:
# Use Rougue score to evaluate the training process
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
  preds, labels = eval_preds

  #decode preds and labels
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #rougeLSum expects newline after each sentence
  decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
  decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

  result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  return result



In [ ]:
# set up training arguments: for GPU with 16GB VRAM
training_args = Seq2SeqTrainingArguments(
    output_dir = "./results",
    evaluation_strategy = "epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    push_to_hub=True
)

In [ ]:
# set up trainer
trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
# train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,0.002300,0.000002,0.605795,0.599609,0.605425,0.606077
2,0.000100,0.000001,0.605795,0.599609,0.605425,0.606077
3,0.000000,0.000000,0.605795,0.599609,0.605425,0.606077


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=5001, training_loss=0.009848088159883054, metrics={'train_runtime': 3074.0043, 'train_samples_per_second': 13.009, 'train_steps_per_second': 1.627, 'total_flos': 1308098638725120.0, 'train_loss': 0.009848088159883054, 'epoch': 3.0})

In [ ]:
trainer.save_model("./t5-drug.ckpt")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

events.out.tfevents.1699796197.3d6399463dea.245.0:   0%|          | 0.00/8.52k [00:00<?, ?B/s]

In [ ]:
import os
os.listdir(".")

['.config', 't5-drug.ckpt', 'results', 'drug_table_v3.xlsx', 'sample_data']

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

print(locale.getpreferredencoding())

UTF-8


In [ ]:
!zip -r t5drug.zip t5-drug.ckpt/

  adding: t5-drug.ckpt/ (stored 0%)
  adding: t5-drug.ckpt/special_tokens_map.json (deflated 85%)
  adding: t5-drug.ckpt/model.safetensors (deflated 8%)
  adding: t5-drug.ckpt/training_args.bin (deflated 51%)
  adding: t5-drug.ckpt/config.json (deflated 63%)
  adding: t5-drug.ckpt/generation_config.json (deflated 29%)
  adding: t5-drug.ckpt/spiece.model (deflated 48%)
  adding: t5-drug.ckpt/added_tokens.json (deflated 83%)
  adding: t5-drug.ckpt/tokenizer_config.json (deflated 95%)


In [ ]:
checkpoint_path = "./t5-drug.ckpt"
model = T5ForConditionalGeneration.from_pretrained(checkpoint_path)
tokenizer_name = T5Tokenizer.from_pretrained(base)


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# actually used it!
import torch

counter = 0  # Initialize a counter

for example in combined_dataset["test"]:
    if counter >= 10:
        break  # Exit the loop after printing the first ten samples

    input_text = example['Question']
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    with torch.no_grad():
        output = model.generate(input_ids, max_length=200)

    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

    print(f"Input text: {input_text}")
    print(f"Generated text: {decoded_output}\n")

    counter += 1  # Increment the counter


Input text: [Rationale] Is fluconazole associated with antifungal medicine and Candida auris?
Generated text: fluconazole is associated with antifungal medicine and is particularly relevant in the context of Candida auris. Fluconazole, belonging to the azole antifungal class, is widely used to inhibit the growth and reproduction of fungal cells. It targets the cytochrome P450 enzyme crucial for ergosterol synthesis, a key component of fungal cell membranes. This disruption weakens the fungal cell membrane, effectively treating and preventing various fungal infections. In the case of Candida auris, a resilient and often drug-resistant fungus, fluconazole is one of the antifungal medications considered for treatment. Its association with both general antifungal medicine and its specific relevance in addressing Candida auris underscores its importance in combating fungal infections.

Input text: [Rationale] Is amphotericin associated with antifungal medicine and Candida auris?
Generated t